Import Libraries:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

In [ ]:
data = pd.read_csv('/kaggle/input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv')
data.head()

In [ ]:
data.shape

In [ ]:
data.dtypes

In [ ]:
data.describe()

In [ ]:
data.isna().any()

In [ ]:
data['Attrition'].value_counts()

In [ ]:
attri_count = data['Attrition'].value_counts()

fig = plt.figure(figsize=(6, 6))
ax = sns.barplot(attri_count.index, attri_count.values)
plt.title("Attrition Distribution",fontsize = 20)
plt.ylabel('Number of Instances', fontsize = 12)
plt.xlabel('Attrition', fontsize = 12);

In [ ]:
data['JobSatisfaction'].value_counts()

Examine relationship between 'Attrition' and 'JobSatisfaction':

In [ ]:
ct = pd.crosstab(data.Attrition, data.JobSatisfaction, margins=True)
ct

Chi-squared test: First, compute both the observation frequency and expected frequency.

In [ ]:
obs = np.append(ct.iloc[0][0:4].values, ct.iloc[1][0:4].values)
obs

In [ ]:
row_sum = ct.iloc[0:2,4].values
exp = []
for j in range(2):
    for val in ct.iloc[2,0:4].values:
        exp.append(val * row_sum[j] / ct.loc['All', 'All'])
exp

In [ ]:
chi_sq_stats = ((obs - exp)**2/exp).sum()
chi_sq_stats

In [ ]:
dof = (len(row_sum)-1)*(len(ct.iloc[2,0:4].values)-1)
dof

In [ ]:
1 - stats.chi2.cdf(chi_sq_stats, dof)

Use SciPy to double check:

In [ ]:
obs = np.array([ct.iloc[0][0:4].values,
                  ct.iloc[1][0:4].values])
stats.chi2_contingency(obs)[0:3]

'Attrition' vs 'WorkLifeBalance'

In [ ]:
ct = pd.crosstab(data.Attrition, data.WorkLifeBalance, margins=True)
ct

In [ ]:
obs = np.array([ct.iloc[0][0:4].values,
                  ct.iloc[1][0:4].values])
stats.chi2_contingency(obs)[0:3]

'Attrition' vs 'Education':

In [ ]:
ct = pd.crosstab(data.Attrition, data.Education, margins=True)
ct

In [ ]:
obs = np.array([ct.iloc[0][0:5].values,
                  ct.iloc[1][0:5].values])
stats.chi2_contingency(obs)[0:3]

Since the p-value is high, we conclude that attrition is independent of education level.

Checking if the dependent relationship between Attrition and Job Satisfaction is agnostic to department:

In [ ]:
dep_counts = data['Department'].value_counts()
dep_counts

In [ ]:
ct = pd.crosstab(data.Attrition, data.Department, margins=True)
ct

In [ ]:
alpha = 0.05
for i in dep_counts.index[0:2]:
    sub_data = data[data.Department == i]
    ct = pd.crosstab(sub_data.Attrition, sub_data.WorkLifeBalance, margins=True)
    obs = np.array([ct.iloc[0][0:4].values,ct.iloc[1][0:4].values])
    print("For " + i + ": ")
    print(ct)
    print('With an alpha value of {}:'.format(alpha))
    if stats.chi2_contingency(obs)[1] <= alpha:
        print("Dependent relationship between Attrition and Work Life Balance")
    else:
        print("Independent relationship between Attrition and Work Life Balance")
    print("")